In [4]:
import pandas as pd
from rich.progress import track

In [10]:
dates = pd.date_range("2020-10-01", "2021-11-01")
data = pd.DataFrame()
for idate in track(dates):
    url = f"https://archive.analytics.mybinder.org/events-{idate:%Y-%m-%d}.jsonl"
    idata = pd.read_json(url, lines=True)
    data = data.append(idata)

Output()

In [11]:
# Prune some columns
KEEP_COLS = ["timestamp", "provider", "spec", "origin"]
data = data[KEEP_COLS]

# Timestamp -> date
data['date'] = data['timestamp'].dt.date
data.drop('timestamp', axis=1, inplace=True)

In [12]:
# Set the timestamps to count number of launches by day
daily = (
    data.groupby(['spec', 'provider', 'origin', 'date']).agg({"spec": "count"}).
    rename(columns={'spec': 'launches'}).reset_index()
)

# Convert back to DT so we can split by month
daily['date'] = pd.to_datetime(daily['date'])

In [13]:
# Save to disk by month
for month, idata in daily.groupby(pd.Grouper(freq='M', key='date')):
    path_data = f"../data/{month:%Y-%m}.json"
    idata.to_json(path_data)